<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [5]:
## Task 1: Mark all launch sites on a map

# Extract unique launch sites and their coordinates from the dataset
launch_sites_data = {
    "CCAFS LC-40": [28.5623, -80.5774],
    "CCAFS SLC-40": [28.5632, -80.5768],
    "KSC LC-39A": [28.5721, -80.5895],
    "VAFB SLC-4E": [34.6328, -120.6106]
}

# Create a new Folium map centered around Florida launch sites
launch_map = folium.Map(location=[28.5, -80.5], zoom_start=6)

# Add markers for each launch site
for site, coords in launch_sites_data.items():
    folium.Marker(
        location=coords,
        popup=site,
        icon=folium.Icon(color="blue", icon="rocket")
    ).add_to(launch_map)

# Display the map
launch_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Add a marker for NASA Johnson Space Center
folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_to(site_map)
folium.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
).add_to(site_map)

# For each launch site, add a Circle object and a Marker with popup labels
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row["Lat"], row["Long"]],
        radius=1000,  # Adjust size to make it visible
        color='#0000FF',
        fill=True,
        fill_opacity=0.6
    ).add_child(folium.Popup(row["Launch Site"])).add_to(site_map)

    folium.Marker(
        location=[row["Lat"], row["Long"]],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#0000FF;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [24]:
# Task 2: Mark the success/failed launches for each site on the map

# Standardize column names to remove any hidden spaces
launch_sites_df.columns = launch_sites_df.columns.str.strip()

# Ensure the column names are properly formatted
if "Launch_Site" not in launch_sites_df.columns:
    launch_sites_df.rename(columns={"Launch Site": "Launch_Site"}, inplace=True)

# Reinitialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a marker for NASA Johnson Space Center
folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_to(site_map)
folium.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
).add_to(site_map)

# Add launch site markers with names
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row["Lat"], row["Long"]],
        radius=1000,
        color='#0000FF',
        fill=True,
        fill_opacity=0.6
    ).add_child(folium.Popup(str(row["Launch_Site"]))).add_to(site_map)

    folium.Marker(
        location=[row["Lat"], row["Long"]],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#0000FF;"><b>{row["Launch_Site"]}</b></div>'
        )
    ).add_to(site_map)

# Function to determine marker color based on class (0 = Failed, 1 = Success)
def get_marker_color(outcome):
    return "green" if outcome == 1 else "red"

# Add mission outcome markers for each launch attempt
for index, row in spacex_df.iterrows():
    lat, lon = row["Lat"], row["Long"]
    
    # Add a marker with color based on success/failure
    folium.Marker(
        location=[lat, lon],
        popup=f"Launch Site: {row['Launch_Site']}\nOutcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=get_marker_color(row["class"]), icon="rocket")
    ).add_to(site_map)

# Display the updated map with success/failure markers
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [25]:
spacex_df.tail(10)

,Launch_Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [26]:
marker_cluster = MarkerCluster()
from folium.plugins import MarkerCluster

# Create a new column in spacex_df to store marker colors based on class value
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")

# Initialize the map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a marker for NASA Johnson Space Center
folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_to(site_map)
folium.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
).add_to(site_map)

# Add launch site markers with names
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row["Lat"], row["Long"]],
        radius=1000,
        color='#0000FF',
        fill=True,
        fill_opacity=0.6
    ).add_child(folium.Popup(str(row["Launch_Site"]))).add_to(site_map)

    folium.Marker(
        location=[row["Lat"], row["Long"]],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#0000FF;"><b>{row["Launch_Site"]}</b></div>'
        )
    ).add_to(site_map)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add mission outcome markers to the MarkerCluster
for index, row in spacex_df.iterrows():
    lat, lon = row["Lat"], row["Long"]
    
    # Add a marker with color based on success/failure inside the cluster
    folium.Marker(
        location=[lat, lon],
        popup=f"Launch Site: {row['Launch_Site']}\nOutcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row["marker_color"], icon="rocket")
    ).add_to(marker_cluster)

# Display the map with clustered launch markers
site_map


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [30]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Apply a function to set marker_color based on the class column
# Apply a function to set marker_color based on the class column
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [31]:
# Add marker_cluster to current site_map

site_map.add_child(marker_cluster)

# Add each launch result as a marker in marker_cluster
for index, record in spacex_df.iterrows():
    # Create and add a Marker object to marker_cluster
    marker = folium.Marker(
        location=[record["Lat"], record["Long"]],
        popup=f"Launch Site: {record['Launch_Site']}\nOutcome: {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=record["marker_color"], icon="rocket")
    )
    marker_cluster.add_child(marker)

# Display the updated map with clustered launch markers
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [35]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Redefine proximity points (nearest city, airport, highway, coastline)
proximity_points = {
    "Nearest City (Orlando, FL)": (28.5383, -81.3792),
    "Nearest Airport (Orlando Int'l Airport)": (28.4312, -81.3081),
    "Nearest Highway (Interstate 95)": (28.5641, -80.8210),
    "Coastline (Cape Canaveral Beach)": (28.3922, -80.6077)
}

# Calculate distances from each launch site to key proximity points
distance_data = []

for index, row in launch_sites_df.iterrows():
    launch_site_name = row["Launch_Site"]
    launch_lat, launch_lon = row["Lat"], row["Long"]
    
    for loc_name, (loc_lat, loc_lon) in proximity_points.items():
        distance_km = haversine(launch_lat, launch_lon, loc_lat, loc_lon)
        distance_data.append({
            "Launch Site": launch_site_name,
            "Proximity Location": loc_name,
            "Distance (km)": round(distance_km, 2)
        })

# Convert to DataFrame and display results
distance_df = pd.DataFrame(distance_data)


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [36]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [37]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [38]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Define the closest coastline coordinates manually (based on Cape Canaveral proximity)
closest_coastline_coords = {
    "CCAFS LC-40": (28.56367, -80.57163),
    "CCAFS SLC-40": (28.56367, -80.57163),
    "KSC LC-39A": (28.573255, -80.646895),
    "VAFB SLC-4E": (34.632834, -120.610745)
}

# Calculate the distance from each launch site to its closest coastline point
coastline_distances = []

for index, row in launch_sites_df.iterrows():
    launch_site = row["Launch_Site"]
    launch_lat, launch_lon = row["Lat"], row["Long"]
    coastline_lat, coastline_lon = closest_coastline_coords[launch_site]

    distance_km = calculate_distance(launch_lat, launch_lon, coastline_lat, coastline_lon)
    coastline_distances.append({
        "Launch Site": launch_site,
        "Coastline Coordinate": f"Lat: {coastline_lat}, Lon: {coastline_lon}",
        "Distance to Coastline (km)": round(distance_km, 2)
    })

# Convert to DataFrame and display results
coastline_distance_df = pd.DataFrame(coastline_distances)



In [39]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
# Add markers for closest coastline points on the map
for index, row in coastline_distance_df.iterrows():
    launch_site_name = row["Launch Site"]
    coastline_coord = closest_coastline_coords[launch_site_name]
    distance = row["Distance to Coastline (km)"]

    # Create and add a marker on the closest coastline point
    distance_marker = folium.Marker(
        location=coastline_coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance:.2f} KM</b></div>'
        )
    )
    site_map.add_child(distance_marker)

# Display the updated map with coastline markers
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [53]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate

 # Reinitialize the map to ensure proper visualization
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# Add Mouse Position for coordinate tracking
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Add launch site markers with names
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row["Lat"], row["Long"]],
        radius=1000,
        color='#0000FF',
        fill=True,
        fill_opacity=0.6
    ).add_to(site_map)

    folium.Marker(
        location=[row["Lat"], row["Long"]],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#0000FF;"><b>{row["Launch_Site"]}</b></div>'
        )
    ).add_to(site_map)

# Add markers for closest coastline points and draw PolyLines
for index, row in coastline_distance_df.iterrows():
    launch_site_name = row["Launch Site"]
    
    # Extract launch site coordinates
    launch_coords = launch_sites_df.loc[launch_sites_df["Launch_Site"] == launch_site_name, ["Lat", "Long"]].values
    if len(launch_coords) > 0:
        launch_coords = [launch_coords[0][0], launch_coords[0][1]]  # Ensure proper list format
    else:
        continue  # Skip if launch site is not found

    # Extract closest coastline coordinates
    coastline_coords = closest_coastline_coords.get(launch_site_name, None)
    if coastline_coords is None:
        continue  # Skip if coastline coordinates are missing

    # Create and add a marker on the closest coastline point with distance label
    distance_marker = folium.Marker(
        location=coastline_coords,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(row["Distance to Coastline (km)"])
        )
    )
    site_map.add_child(distance_marker)

    # Create a PolyLine between the launch site and the closest coastline point
    lines = folium.PolyLine(locations=[launch_coords, list(coastline_coords)], weight=3, color="blue")
    site_map.add_child(lines)

# Display the updated map with markers and PolyLines
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [54]:
# Define additional proximity points for closest city, railway, and highway
additional_proximity_points = {
    "Nearest City (Orlando, FL)": (28.5383, -81.3792),
    "Nearest Railway (Florida East Coast Railway)": (28.5721, -80.5853),
    "Nearest Highway (Interstate 95)": (28.5641, -80.8210),
}

# Add markers for additional proximity points and draw PolyLines to launch sites
for index, row in launch_sites_df.iterrows():
    launch_site_name = row["Launch_Site"]
    launch_coords = (row["Lat"], row["Long"])

    for loc_name, loc_coords in additional_proximity_points.items():
        # Calculate distance using Haversine formula
        distance_km = calculate_distance(launch_coords[0], launch_coords[1], loc_coords[0], loc_coords[1])

        # Create and add a marker for proximity points with distance label
        distance_marker = folium.Marker(
            location=loc_coords,
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_km)
            )
        )
        site_map.add_child(distance_marker)

        # Create a PolyLine between the launch site and the proximity point
        lines = folium.PolyLine(locations=[launch_coords, loc_coords], weight=2, color="green")
        site_map.add_child(lines)

# Display the updated map with additional markers and PolyLines
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
